# With ChatModel

In [ ]:
import json
import helper
import pandas as pd

from pprint import pprint

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser



In [ ]:
def save_result(algorithm, output, prompt_type, model, init, best_practice, format_prompt, output_parser, ticketUri, ticket, reruns=0, evalMode=False):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice,
        'formatter_uri': format_prompt}
    result['output_parser'] = output_parser
    result['input_data'] = {
        'ticket_uri': ticketUri,
        'jira': ticket['Jira'],
        'id': ticket['IssueId'],
        'evolution': ticket['EvoId']}
    result['reruns'] = reruns
    result['output'] = output

    helper.annotateResult(result, ticket, prompt_type)

    output_name = 'output_'+date+'_'+time+'.json'

    if evalMode:
        with open('evaluation/' + prompt_type + '/' + model + '/' + algorithm + "/" + output_name, 'w') as f:
            json.dump(result, f, indent=4)
        print('Evaluation mode: result saved in evaluation folder')
    else:
        with open('results/' + prompt_type + '/' + output_name, 'w') as f:
            json.dump(result, f, indent=4)        

In [ ]:
MODELNAME = 'gpt-4-0125-preview'

model = ChatOpenAI(
    model=MODELNAME,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

# Hier wird das Ticket noch über der init_prompt eingespielt
# INITPROMPT = 'initPrompt_V1.3.0.json'

# Hier wird das Ticket über der best_practice_prompt eingespielt
INITPROMPT = 'initPrompt_V2.1.0.json'

## Summary Length

#### 0-Shot

In [ ]:
SUMMARYPROMPT = 'summaryLengthPrompt_V2.4.0.json'
SUMMARYFORMATPROMPT = 'summaryLengthFormatPrompt_V1.5.0.json'

In [ ]:
async def runDetectionOfSummaryLength(ticketUri, ticket, reruns= 0, evalMode=False):
    ### Set variables ###
    minChars = 39
    maxChars = 70

    init_prompt = load_prompt('prompts/init/' + INITPROMPT)
    bestPractice_prompt = load_prompt('prompts/summary/' + SUMMARYPROMPT).format(min=minChars, max=maxChars)
    format_prompt = load_prompt('prompts/summary/' + SUMMARYFORMATPROMPT)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )

    ### Run chains ###
    output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":ticket})

    ### Save result ###
    save_result('0Shot', output, 'summary', MODELNAME, INITPROMPT, SUMMARYPROMPT, SUMMARYFORMATPROMPT, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
annotatedDataset = pd.read_csv('data/summary/summaryDataset.csv')

amount = annotatedDataset.shape[0]
num = 1
errorCounter = 0

for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/summary/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"
    
    with open(ticketUri) as f:
        ticket = json.load(f)
    try:
        print("######## (" + str(num) + "/" + str(errorCounter) + "/" + str(amount) + ") Running ticket: " + ticketUri + " ########")
        print("     ")
        await runDetectionOfSummaryLength(ticketUri, ticket, 1, True)
        print("     ")
    except:
        print('Error with ticket: ' + ticketUri)
        errorCounter += 1
        continue
    finally:
        num += 1

#### Few-Shot

In [ ]:
SUMMARYPROMPT = 'summaryLengthPrompt_FewShot_V2.3.0.json'
SUMMARYFORMATPROMPT = 'summaryLengthFormatPrompt_V1.5.0.json'

In [ ]:
async def runDetectionOfSummaryLength(ticketUri, ticket, reruns= 0, evalMode=False):
    ### Set variables ###
    minChars = 39
    maxChars = 70

    init_prompt = load_prompt('prompts/init/' + INITPROMPT)
    bestPractice_prompt = load_prompt('prompts/summary/' + SUMMARYPROMPT).format(min=minChars, max=maxChars, ticket=ticket)
    format_prompt = load_prompt('prompts/summary/' + SUMMARYFORMATPROMPT)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )
    
    ### Run chains ###
    output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt})

    ### Save result ###
    save_result('FewShot', output, 'summary', MODELNAME, INITPROMPT, SUMMARYPROMPT, SUMMARYFORMATPROMPT, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
annotatedDataset = pd.read_csv('data/summary/summaryDataset.csv')

amount = annotatedDataset.shape[0]
num = 1
errorCounter = 0

for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/summary/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"
    
    with open(ticketUri) as f:
        ticket = json.load(f)
    try:
        print("######## (" + str(num) + "/" + str(errorCounter) + "/" + str(amount) + ") Running ticket: " + ticketUri + " ########")
        print("     ")
        await runDetectionOfSummaryLength(ticketUri, ticket, 0, True)
        print("     ")
    except:
        print('Error with ticket: ' + ticketUri)
        errorCounter += 1
        continue
    finally:
        num += 1

#### 0-Shot CoT

In [ ]:
SUMMARYPROMPT = 'summaryLengthPrompt_0ShotCoT_V1.0.0.json'
SUMMARYFORMATPROMPT = 'summaryLengthFormatPrompt_V1.4.0.json'

In [ ]:
async def runDetectionOfSummaryLength(ticketUri, ticket, reruns= 0, evalMode=False):
    ### Set variables ###
    minChars = 39
    maxChars = 70

    init_prompt = load_prompt('prompts/init/' + INITPROMPT)
    bestPractice_prompt = load_prompt('prompts/summary/' + SUMMARYPROMPT).format(min=minChars, max=maxChars)
    format_prompt = load_prompt('prompts/summary/' + SUMMARYFORMATPROMPT)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )

    ### Run chains ###
    output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":ticket})

    ### Save result ###
    save_result('0ShotCoT', output, 'summary', MODELNAME, INITPROMPT, SUMMARYPROMPT, SUMMARYFORMATPROMPT, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
annotatedDataset = pd.read_csv('data/summary/summaryDataset.csv')

amount = annotatedDataset.shape[0]
num = 1
errorCounter = 0

for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/summary/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"
    
    with open(ticketUri) as f:
        ticket = json.load(f)
    try:
        print("######## (" + str(num) + "/" + str(errorCounter) + "/" + str(amount) + ") Running ticket: " + ticketUri + " ########")
        print("     ")
        await runDetectionOfSummaryLength(ticketUri, ticket, 2, True)
        print("     ")
    except:
        print('Error with ticket: ' + ticketUri)
        errorCounter += 1
        continue
    finally:
        num += 1

## Arbitrary Structure

In [ ]:
def runDetectionOfArbitraryStructure(init_prompt_name, bestPractice_prompt_name, format_prompt_name, ticketUri, ticket, issue_type, desired_structure, structure_desc, reruns= 0, evalMode=False):
    ### Set Prompts ### 
    init_prompt = load_prompt('prompts/init/'+init_prompt_name)
    bestPractice_prompt = load_prompt('prompts/arbitraryStructure/'+bestPractice_prompt_name).format(issue_type=issue_type, structure_desc=structure_desc, structure=desired_structure)
    format_prompt = load_prompt('prompts/arbitraryStructure/'+format_prompt_name)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )

    ### Run chains ###
    output = chain2.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":ticket})

    ### Save result ###
    save_result(output, 'summary', MODELNAME, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
issue_type = "User Story"
desired_structure = "As a <Role>, I want/want to/need/can/would <Task>, so that <Goal>"
structure_desc = """Role: abstract behavior of actors in the system context; describes who uses the system.
Task: specific things that must be done to achieve goals; solution or function of the problem.
Goal: a condition or a circumstance desired by stakeholders or actors; describes the problem domain or the impact of solving the problem."""
# structure_desc = """Role: abstract behavior of actors in the system context; describes who uses the system.
# Task: specific things that must be done to achieve goals; solution or function of the problem.
# Goal: a condition or a circumstance desired by stakeholders or actors; describes the problem domain or the impact of solving the problem.
# Capability: the ability of actors to achieve goals based on certain conditions and events."""

In [ ]:
annotatedDataset = pd.read_csv('data/arbitraryStructure/arbitraryStructureDataset.csv')

for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/arbitraryStructure/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"

    with open(ticketUri) as f:
        json_sample = json.load(f)
    
    try:
        runDetectionOfArbitraryStructure('initPrompt_V1.3.0.json', 'arbitraryStructurePrompt_V1.0.0.json', 'arbitraryStructureFormatPrompt_V1.0.0.json', ticketUri, json_sample, issue_type, desired_structure, structure_desc)
    except:
        print("Error with ticket: " + ticketUri)



## Field Update

In [ ]:
UPDATEPROMPT = 'updatePrompt_V1.26.0.json'
UPDATEFORMATPROMPT = 'updateFormatPrompt_V1.11.0.json'

In [ ]:
async def runDetectionOfOutdatedFields(ticket, ticketUri, reruns= 0, evalMode=False):
    ### Set variables ###
    init_prompt = load_prompt('prompts/init/' + INITPROMPT)
    bestPractice_prompt = load_prompt('prompts/update/' + UPDATEPROMPT).format()
    format_prompt = load_prompt('prompts/update/' + UPDATEFORMATPROMPT)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )

    ### Run chains ###
    output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":ticket})

    # ### Save result ###
    save_result(output, 'update', MODELNAME, INITPROMPT, UPDATEPROMPT, UPDATEFORMATPROMPT, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
annotatedDataset = pd.read_csv('data/update/updateDataset.csv')

amount = annotatedDataset.shape[0]
num = 1
for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/update/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"
    
    with open(ticketUri) as f:
        ticket = json.load(f)
    try:
        print("######## (" + str(num) + "/" + str(amount) + ") Running ticket: " + ticketUri + " ########")
        print("     ")
        await runDetectionOfOutdatedFields(ticket, ticketUri, 1, True)
        print("     ")
    except Exception as e:
        print('Error with ticket: ' + ticketUri)
        print(e)
        continue
    finally:
        num += 1

## Toxic Speech Detection

In [ ]:
sample_name = "last_row_toxic.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

format_prompt_name = 'toxicSpeechFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/toxicSpeech/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [ ]:
output = await chain2.ainvoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [ ]:
save_result(output, 'toxicSpeech', MODELNAME, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)

## Bug Report Structure

In [ ]:
BUGREPORTPROMPT = 'bugReportStructurePrompt_V2.3.0.json'

In [ ]:
sample_name = "Jira_1388024_0.json"
with open('./data/bugreportStructure/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+BUGREPORTPROMPT)

In [ ]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [ ]:
output = await chain.ainvoke({"bug_report":json_sample})

In [ ]:
save_result(output, 'bugReportStructure', MODELNAME, '-', BUGREPORTPROMPT, '-', 'StrOutputParser', sample_name, json_sample)

## Internationalization

In [ ]:
sample_name = "last_row_german.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'internationalizationPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/internationalization/'+bestPractice_prompt_name).format()

format_prompt_name = 'internationalizationFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/internationalization/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()

chain2 = (
    {"revised_ticket":chain1}
    | format_prompt
    | model
    | JsonOutputParser()
    )

In [ ]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [ ]:
save_result(output, 'internationalization', MODELNAME, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)